In [1]:
import torch
import torchvision
import cv2
import numpy as np


ModuleNotFoundError: No module named 'torch'

In [ ]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

model.load_state_dict(torch.load('best_model.pth'))

device = torch.device('cuda')
model = model.to(device)

In [ ]:
import math
from time import sleep
def convert_angle(x,y):
    if(y == 0):      #special case when y = 0 
        if(x == 0):          #reach the destination
            return None
        elif(x > 0):     
            return 90
        else:
            return -90
    temp = x/y
    if(y > 0): #in case from -90 to 90 degree
        return math.degrees(math.atan(temp))
    
    else:
        if(x > 0):  #case for 90 to 180 degree
            return 180 - math.degrees(math.atan(temp))
        else:       #case for -180 to -90 degree
            return math.degrees(math.atan(temp)) - 180
        
def update_info_straight(x,y,curr_angle, time):    #for keep space
    speed = 1       #currently defalut 1, can use calibration to change that
    x = x + math.sin(math.radians(curr_angle)) * time * speed
    y = y + math.cos(math.radians(curr_angle)) * time * speed

    return True

def reach_destination(curr_x, x, curr_y, y):  
    distance_apart = math.sqrt((curr_x - x)**2 + (curr_y - y)**2)       #use to calculate the distance from the destination to current lcoation
    
    if(distance_apart < 1): #within a range, then it mean it is close to the destination, then it return true.
        return True
    else:
        return False

def turn_left(leftspeed = 0.4, rightspeed = 0.6):    # use for easy control of the car
    robot.set_motors(leftspeed, rightspeed)

def turn_right(leftspeed = 0.6, rightspeed = 0.4):    
    robot.set_motors(leftspeed, rightspeed)

def spin_left(speed = 0.5):
    robot.left(speed)

def spin_right(speed = 0.5):
    robot.right(speed)

def straight(speed = 1):
    robot.forward(speed)

def check_angle(angle):     #use to convert the angle to be between -180 to 180 degree
    if(angle > 180):
        return -180 + angle - 180
    elif(angle < -180):
        return 180 + angle + 180
    else:
        return angle

In [ ]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)
display(widgets.HBox([image, blocked_slider]))

In [ ]:
from jetbot import Robot

robot = Robot()
robot.stop()

curr_x = 0          #use to keep track on the x of the car current status
curr_y = 0          #use to keep track on the y of the car current status
curr_angle = 0      #use to keep track on the angle of the car current status, range from -180 to 180. 0 indicate straight
des_x = None
des_y = None

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot, curr_x, curr_y, curr_angle 
    global des_x, des_y
    
    if(reach_destination(curr_x, x, curr_y, y)):
        break
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5: #if no object
        target_angle = convert_angle(x - curr_x, y - curr_y)
        angle_different = curr_angle - target_angle
            if(abs(angle_different) > 2):#not pointing at the correct angle
                #adjust to the left while going straight if angle is negative and right if angle is positive
                if(angle_different < 0):
                    robot.left(0.5)
                else:
                    robot.right(0.5)

                time_to_turn = spin_time / abs(angle_different)     #calculate how long does the spin need to be

                sleep(time_to_turn)     #sleep to wait until the car turn enough
                curr_angle = target_angle    #x and y did not change, only the angle change to the target value in this case.
                
            else:
                #go straight 
                straight()
                sleep(0.1)
                update_info_straight(curr_x,curr_y,curr_angle, 0.1)            
            
    else:
        spin_left()
        sleep(0.1)
        curr_angle = check_angle(curr_angle - (time_to_turn/0.1) * 360)  #make sure the angle is between -180 to 180 degree
        straight()
        sleep(0.5)
        update_info_straight(curr_x, curr_y,curr_angle,0.5)
        
    time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

In [ ]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

In [ ]:
camera.unobserve(update, names='value')
robot.stop()